In [ ]:
!git clone https://github.com/jayded/evidence-inference.git

In [1]:
from process_evidence_inference import read_docs

In [2]:
docs = read_docs(abst_only=True)

Reading prompts + articles
2419
10137
Processing annotations
Retained 7193/19716 valid annotations (613 w/ bad offsets)
Retained 1772/2419 docs with nonzero prompts


In [8]:
from bisect import *
def find_ge(a, x):
    'Find leftmost item greater than or equal to x'
    i = bisect_left(a, x)
    return i
    
def find_le(a, x):
    'Find rightmost value less than or equal to x'
    i = bisect_right(a, x)
    if i:
        return i-1
    raise ValueError

In [9]:
import spacy
nlp = spacy.load("en_core_web_sm")

In [23]:
from tqdm import tqdm
annotations = []
for d in tqdm(docs) :
    dn = nlp(d.text)
    text = " ".join([t.text for t in dn if t.text.strip() != ''])
    tidx = [t.idx for t in dn if t.text.strip() != '']
    for i, f in enumerate(d.frames) :
        annotation_id = str(d.id) + '_' + str(i)
        annotations.append({
            'annotation_id' : annotation_id,
            'document' : text,
            'rationale' : [(find_le(tidx, f.ev.i), find_ge(tidx, f.ev.f))],
            'label' : f.encoder_label[f.label],
            'query' : ' || '.join([f.i, f.c, f.o])
        })

100%|██████████| 1772/1772 [01:13<00:00, 23.97it/s]


In [32]:
train_ids = [int(x) for x in open('evidence-inference/annotations/splits/train_article_ids.txt').read().strip().split()]
dev_ids = [int(x) for x in open('evidence-inference/annotations/splits/validation_article_ids.txt').read().strip().split()]
test_ids = [int(x) for x in open('evidence-inference/annotations/splits/test_article_ids.txt').read().strip().split()]

In [36]:
import json
with open('data/train.jsonl', 'w') as g :
    g.write('\n'.join([json.dumps(ann) for ann in annotations if int(ann['annotation_id'].split('_')[0]) in train_ids]))

In [37]:
with open('data/dev.jsonl', 'w') as g :
    g.write('\n'.join([json.dumps(ann) for ann in annotations if int(ann['annotation_id'].split('_')[0]) in dev_ids]))

In [38]:
with open('data/test.jsonl', 'w') as g :
    g.write('\n'.join([json.dumps(ann) for ann in annotations if int(ann['annotation_id'].split('_')[0]) in test_ids]))